<img src='./img/EU-Copernicus-EUM_3Logos.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='50%'></img>


<br>

<a href="./11_ltpy_atmospheric_composition_overview.ipynb"><< 11 - Atmospheric Composition Data - Overview</a><span style="float:right;"><a href="./211_ltpy_AC_SAF_GOME-2_L2_load_browse.ipynb">211 - AC SAF GOME-2 Level 2 Load and Browse>></a></span>

# 1.2 WEkEO Harmonized Data Access (HDA) API - How-To

[WEkEO](https://www.wekeo.eu/) is the EU Copernicus DIAS (Data and Information Access Service) reference service for environmental data, virtual processing environments and skilled user support.

WEkEO offers access to a variety of data, including different parameters sensored from Sentinel-1, Sentinel-2 and Sentinel-3. It further offers access to climate reanalysis and seasonal forecast data.

The [Harmonised Data Access (HDA) API](https://www.wekeo.eu/documentation/using_jupyter_notebooks), a REST interface, allows users to subset and download datasets from WEkEO.

This notebook is a step-by-step guide on how to search for and download data from WEkEO using the `HDA API`. The notebook makes use of functions stored in the notebook [hda_api_functions](./hda_api_functions.ipynb).

Follow the following steps:

 - [1. Search for datasets on WEkEO](#wekeo_search)
 - [2. Get the dataset's Collection ID](#wekeo_collection_id)
 - [3. Get your WEkEO API key](#wekeo_api_key)
 - [4. Initialise the WEkEO Harmonised Data Access request](#wekeo_hda_request)
 - [5. Load data descriptor file and request data](#wekeo_json)
 - [6. Download requested data](#wekeo_download)

#### Load required libraries

In [ ]:
import os
import sys
import json
import time
import base64

import requests
import warnings
warnings.filterwarnings('ignore')

#### Load helper functions

In [ ]:
from ipynb.fs.full.hda_api_functions import *

<hr>

### <a id='wekeo_search'></a>1. Search for datasets on WEkEO

Under [WEkEO DATA](https://www.wekeo.eu/web/guest/data), you can search all datasets available on WEkEO. There are two search options:<br> 
- a `free keyword search`, and 
- an `advanced search`, that opens a selection menu on the left to help navigating the search.<br> 

You can look for example for *`sentinel 3 level 1`* and browse through the results. You can click on the dataset you are interested in and you will be guided to a dataset description.

<br>

<div style='text-align:center;'>
<figure><img src='./img/wekeo_data_search.png' width='70%' />
    <figcaption><i>WEkEO interface to search for datasets</i></figcaption>
</figure>
</div>

### <a id='wekeo_collection_id'></a>2. Get the Collection ID 

The dataset description provides the following information:
- **General description** of the dataset,
- How to **access** the data, 
- **Additional resources**, such as a link to the Product Data Format Specification guide, 
- **Contact**, 
- **Technical details**, and 
- **Additional information**. 

Under `Additional information`, you can look up the `Collection ID`. You need the `Collection ID` to request data from the Harmonised Data Access API. 

For `OLCI Level 1B Full Resolution - Sentinel-3` data for example, the collection ID is `EO:EUM:DAT:SENTINEL-3:OL_1_EFR___`.

<br>

<div style='text-align:center;'>
<figure><img src='./img/wekeo_collection_id.png' width='60%' />
    <figcaption><i>Additional dataset information on WEkEO</i></figcaption>
</figure>
</div>
<br>

Let's store the Collection ID as a variable called `dataset_id` to be used later.

In [ ]:
dataset_id = "EO:EUM:DAT:SENTINEL-3:OL_1_EFR___"

### <a id='wekeo_api_key'></a>3. Get the WEkEO API key

In order to interact with WEkEO's Harmonised Data Access API, each user gets assigned an `API key` and `API token`. You will need the API key in order to download data in a programmatic way.

The `api key` is generated by encoding your `username` and `password` to Base64. You can use the function [generate_api_key](./hda_api_functions.ipynb#generate_api_key) to programmatically generate your Base64-encoded api key. For this, you have to replace the 'username' and 'password' strings with your WEkEO username and password in the cell below.

Alternatively, you can go to this [website](https://www.base64encode.org/) that allows you to manually encode your `username:password` combination. An example of an encoded key is `wekeo-test:wekeo-test`, which is encoded to `d2VrZW8tdGVzdDp3ZWtlby10ZXN0`.


In [ ]:
user_name = 'xxxxxxxxxxxxx'
password = 'xxxxxxxxxx'

In [ ]:
api_key = generate_api_key(user_name, password)
api_key

#### Alternative: enter manually the generated api key

In [ ]:
#api_key = 

### <a id='wekeo_hda_request'></a>4. Initialise the Harmonised Data Access (HDA) API request

In order to initialise an API request, you have to initialise a dictionary that contains information on `dataset_id`, `api_key` and `download_directory_path`.

Please enter the path of the directory where the data shall be downloaded to.

In [ ]:
# Enter here the directory path where you want to download the data to
download_dir_path = './'

With `dataset_id`, `api_key` and `download_dir_path`, you can initialise the dictionary with the function [init](./hda_api_functions.ipynb#init).

In [ ]:
hda_dict = init(dataset_id, api_key, download_dir_path)

#### Request access token

Once initialised, you can request an access token with the function [get_access_token](./hda_api_functions.ipynb#get_access_token). The access token is stored in the `hda_dict` dictionary.

You might need to accept the Terms and Conditions, which you can do with the function [acceptTandC](./hda_api_functions.ipynb#acceptTandC).

In [ ]:
hda_dict = get_access_token(hda_dict)

#### Accept Terms and Conditions (if applicable)

In [ ]:
hda_dict = acceptTandC(hda_dict)

### <a id='wekeo_json'></a>5. Load data descriptor file and request data

The Harmonised Data Access API can read your data request from a `JSON` file. In this JSON-based file, you can describe the dataset you are interested in downloading. The file is in principle a dictionary. Following keys can be defined:
- `datasetID`: the dataset's collection ID
- `stringChoiceValues`: type of dataset, e.g. 'Non Time Critical'
- `dataRangeSelectValues`: time period you would like to retrieve data
- `boundingBoxValues`: optional to define a subset of a global field

See an example of a `data descriptor` file [here](./olci_data_descriptor.json)

You can load the `JSON` file with `json.load()`. Alternatively, you can copy paste the dictionary describing your data into a cell.

In [ ]:
with open('./olci_data_descriptor.json', 'r') as f:
    data = json.load(f)
data

In [ ]:
data = {
    "datasetId": "EO:EUM:DAT:SENTINEL-3:OL_1_EFR___",
    "stringChoiceValues": [
        {
            "name": "timeliness",
            "value": "Non Time Critical"
        }
    ],
    "dateRangeSelectValues": [
        {
            "name": "dtrange",
            "start": "2019-07-28T05:50:00.000Z",
            "end": "2019-07-28T06:00:00.000Z"
        }
    ],
    "boundingBoxValues": [
        {
            "name": "bbox",
            "bbox": [
                40,
                -10,
                180,
                90
            ]
        }
    ]
}

#### Initiate the request by assigning a job ID

The function [get_job_id](./hda_api_functions.ipynb#get_job_id) will launch your data request and your request is assigned a `job ID`.

In [ ]:
hda_dict = get_job_id(hda_dict,data)

#### Build list of file names to be ordered and downloaded

The next step is to gather a list of file names available, based on your assigned `job ID`. The function [get_results_list](./hda_api_functions.ipynb#get_results_list) creates the list.

In [ ]:
hda_dict = get_results_list(hda_dict)

#### Create an `order ID` for each file to be downloaded

The next step is to create an `order ID` for each file name to be downloaded. You can use the function [get_order_ids](./hda_api_functions.ipynb#get_order_ids).

In [ ]:
hda_dict = get_order_ids(hda_dict)

### <a id='wekeo_download'></a>6. Download requested data

As a final step, you can use the function [download_data](./hda_api_functions.ipynb#download_data) to initialize the data download and to download each file that has been assigned an `order ID`. 

In [ ]:
download_data(hda_dict)

<br>

<a href="./11_ltpy_atmospheric_composition_overview.ipynb"><< 11 - Atmospheric Composition Data - Overview</a><span style="float:right;"><a href="./211_ltpy_AC_SAF_GOME-2_L2_load_browse.ipynb">211 - AC SAF GOME-2 Level 2 Load and Browse>></a></span>

<hr>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/atmosphere/atmosphere">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:training@eumetsat.int>Contact</a></span></p>